# qBraid Lab GPU-accelerated VQE Demo

## 2D Ising Model: $H=\sum\limits_{(i,j)} J_{ij} Z_i Z_j + \sum\limits_{i} h_i Z_i$

##### Based on https://quantumai.google/cirq/experiments/variational_algorithm

##### First, make sure you have installed the Cirq environment and are using the Cirq python kernel.

In [ ]:
import cirq, qsimcirq, time, random
import numpy as np

##### Randomly initialize the necessary Hamiltonian coefficients

In [ ]:
def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length - 1, length)
    # links within a column
    jc = rand2d(length, length - 1)
    return (h, jr, jc)

h, jr, jc = random_instance(3)

In [ ]:
print(h)
print(jr)
print(jc)

##### Generate the ansatz

In [ ]:
def prepare_plus_layer(length):
    for i in range(length):
        for j in range(length):
            yield cirq.H(cirq.GridQubit(i, j))

In [ ]:
circuit = cirq.Circuit()
circuit.append(prepare_plus_layer(2))
print(circuit)

In [ ]:
def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""

    # Define the gate once and then re-use it for each Operation.
    rot = cirq.XPowGate(exponent=half_turns)

    # Create an X rotation Operation for each qubit in the grid.
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

In [ ]:
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2,0.1))
print(circuit)

In [ ]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

In [ ]:
circuit = cirq.Circuit()
circuit.append(rot_z_layer(h,0.1))

In [ ]:
print(circuit)

In [ ]:
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    cz_gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i + 1, j)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i, j + 1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

In [ ]:
circuit = cirq.Circuit()
circuit.append(rot_11_layer(jr,jc,0.1))

In [ ]:
print(circuit)

In [ ]:
length = 5
qubits = cirq.GridQubit.square(length)
h, jr, jc = random_instance(length)

circuit = cirq.Circuit()
circuit.append(prepare_plus_layer(length))
circuit.append(rot_z_layer(h,0.3))
circuit.append(rot_11_layer(jr,jc,0.3))
circuit.append(rot_x_layer(length, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))

In [ ]:
print(circuit)

##### Let's define the CPU and GPU simulators

In [ ]:
# Default CPU simulator
simulator = cirq.Simulator()

# cuQuantum GPU simulator
ngpus = 1
qsim_options = qsimcirq.QSimOptions(
    max_fused_gate_size = 2,
    use_gpu = True,
    gpu_mode = 1 # gpu_mode: use CUDA if set to 0 (default value) or use the NVIDIA cuStateVec library if set to any other value.
)

simulator_cuQ=qsimcirq.QSimSimulator(qsim_options)

##### Define function for extracting energy measurements

In [ ]:
def energy_func(length, h, jr, jc):
    def energy(measurements):
        # Reshape measurement into array that matches grid shape.
        meas_list_of_lists = [measurements[i * length:(i + 1) * length]
                              for i in range(length)]
        # Convert true/false to +1/-1.
        pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

        tot_energy = np.sum(pm_meas * h)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
        return tot_energy
    return energy

##### Define the expectation value of the energy

In [ ]:
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(length, h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions

##### Run the CPU VQE

In [ ]:
# Default CPU simulator VQE

start = time.time()
results = simulator.run(circuit, repetitions=10000)
# print(results.histogram(key='x'))
cpu_time = time.time() - start
print("CPU runtime: {:.24}s".format(cpu_time))
print(f'Simulated ground-state energy expectation value is {obj_func(results)}')

##### Run the GPU VQE

In [ ]:
# cuQuantum GPU simulator VQE

start = time.time()
results = simulator_cuQ.run(circuit, repetitions=10000)
# print(results.histogram(key='x'))
gpu_time = time.time() - start
print("GPU runtime: {:.24}s".format(gpu_time))
print(f'Simulated ground-state energy expectation value is {obj_func(results)}')

##### Calculate the GPU vs. CPU speedup

In [ ]:
print("GPU VQE was " + str(round(cpu_time/gpu_time,3)) + "X faster than CPU VQE")